In [1]:
#https://github.com/chris-stellato/capstone3/tree/main/notebooks


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2
import math
import random
import os
import time
from tensorflow import keras

In [3]:
convert_to_dt = ['Date']

data = pd.read_csv('C:/Users/tomwr/Datasets/Tabular/tesco-stock.csv',
                   parse_dates = convert_to_dt,
                   infer_datetime_format = True
                  )
data_normal_index = data
data_date = data.set_index('Date')
print(data.head())
print()
print(data.info())

        Date        Open        High         Low       Close   Adj Close  \
0 2021-09-20  254.850006  255.751007  252.050003  254.449997  244.151596   
1 2021-09-21  255.550003  257.850006  254.750000  257.200012  246.790314   
2 2021-09-22  258.750000  261.649994  258.049988  261.649994  251.060181   
3 2021-09-23  263.500000  264.520996  259.500000  259.549988  249.045166   
4 2021-09-24  260.100006  260.500000  257.500000  257.500000  247.078156   

     Volume  
0  13732928  
1  18373464  
2  15194454  
3  13317828  
4   9898053  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       252 non-null    datetime64[ns]
 1   Open       252 non-null    float64       
 2   High       252 non-null    float64       
 3   Low        252 non-null    float64       
 4   Close      252 non-null    float64       
 5   Adj Close  252 non-n

In [4]:
#windowize function, three dimensional output for X and y, split afterwards
#the below windowize functions are imported but pasted here for working through setup
def windowize_data(data, n_prev, y_var, predict_steps=365):
    n_predictions = len(data) - n_prev
    
    print(f'num_predictions: {n_predictions}')
    y = data[y_var].iloc[n_prev:].values
    y_indices = np.arange(predict_steps) + np.arange(len(y) - predict_steps+1)[:, None] # (0 to 365) + (
    y = y[y_indices]
    x_indices = np.arange(n_prev) + np.arange(n_predictions- predict_steps+1)[:, None]
    x = data.values[x_indices]
    return x, y

In [5]:
n_prev = 25 # days behind
n_forecast = 10 # days forecast ahead

X, y = windowize_data(data, n_prev, 'Close', n_forecast)

print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

num_predictions: 227
X shape: (218, 25, 7)
y shape: (218, 10)


In [14]:
WINDOW_SIZE = 30
FORECAST_LEN = 10

def windowize_data2(data, window_size, forecast_len, target):
    target_col = data[target]
    X = []
    y = []
    data_np = data.to_numpy()
    
    for i in range(len(data) - window_size): # for i in range (252 - 25 = 227)
        current_row_index = i - 1
        
        features = [[a] for a in data_np[i : i + window_size]] # row for row in data_numpy[1 : 1 + window_size]
        X.append(features)
        
        #labels = [[b] for b in target_col.iloc[i: i + window_size] in data[i: i + forecast_len]]
        
        #Labels need to be arrays for the next forecast_len values of data[target_col] for each row in data
        
        
        #y.append(labels)
        
    return np.array(X), np.array(y)

X2, y2 = windowize_data2(data_date, WINDOW_SIZE, FORECAST_LEN, 'Close')

In [15]:
print(X2.shape)
print(y2.shape)

(222, 30, 1, 6)
(0,)


In [23]:
print(data_date.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 252 entries, 2021-09-20 to 2022-09-20
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       252 non-null    float64
 1   High       252 non-null    float64
 2   Low        252 non-null    float64
 3   Close      252 non-null    float64
 4   Adj Close  252 non-null    float64
 5   Volume     252 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 13.8 KB
None


In [106]:
#https://www.kaggle.com/code/rinichristy/bi-uni-lstm-rnns-for-temperature-forecasting
#assumes data has been scaled before this stage
def windowize_data3(data, window_size, forecast_len):
    X3 = []
    y3 = []
    for i in range(len(data) - window_size - forecast_len):
        X3.append(data[i: (i + window_size)])
        #y3.append(data[:, -1][i + window_size: i + window_size + forecast_len])
        
        y3.append(data[: -1][i + window_size: i + window_size + forecast_len])

    return np.array(X3), np.array(y3)
    ry3 = y3[:,:,3]
    return np.array(ry3)
X3, y3 = windowize_data3(data_date, WINDOW_SIZE, FORECAST_LEN)
#print(X3.shape)
print(y3.shape) # Needs to be (212, 10) - 212 samples of the next 10 values of close.
print()
print()
print(y3)
print()
print()
#ry3 = [element[3][3] for element in y3]
print(ry3)
print(ry3.shape)
#ry3 = np.array(ry3)
#print(ry3.shape)

#https://stackoverflow.com/questions/52572391/extract-nth-value-from-3d-array (for below)

#ry3 = y3[:,:,3]
#print(ry3)
#ry3_np = np.array(ry3)
#print(ry3_np.shape)

(212, 10, 6)


[[[2.70899994e+02 2.72350006e+02 2.68350006e+02 2.71950012e+02
   2.64080719e+02 1.99486390e+07]
  [2.72450012e+02 2.73083008e+02 2.71200012e+02 2.72000000e+02
   2.64129272e+02 2.85055010e+07]
  [2.72000000e+02 2.74049988e+02 2.70899994e+02 2.73549988e+02
   2.65634399e+02 9.49716700e+06]
  ...
  [2.78649994e+02 2.87399994e+02 2.78602997e+02 2.84950012e+02
   2.76704559e+02 2.88083570e+07]
  [2.84950012e+02 2.84950012e+02 2.81399994e+02 2.83700012e+02
   2.75490723e+02 1.18646360e+07]
  [2.84049988e+02 2.85425995e+02 2.83399994e+02 2.85000000e+02
   2.76753082e+02 1.48182270e+07]]

 [[2.72450012e+02 2.73083008e+02 2.71200012e+02 2.72000000e+02
   2.64129272e+02 2.85055010e+07]
  [2.72000000e+02 2.74049988e+02 2.70899994e+02 2.73549988e+02
   2.65634399e+02 9.49716700e+06]
  [2.73700012e+02 2.77299988e+02 2.73649994e+02 2.75600006e+02
   2.67625092e+02 1.21118540e+07]
  ...
  [2.84950012e+02 2.84950012e+02 2.81399994e+02 2.83700012e+02
   2.75490723e+02 1.18646360e+07]
 